In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
eventsData = pd.read_csv("data/Eventos(+completo).csv",low_memory = False)

In [3]:
#Funcion que pasa a formato datetime las fechas que recibimos.
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

eventsData["timestamp"]= datetime(eventsData["timestamp"],"%Y-%m-%dT%H:%M:%S")

In [4]:
eventsData["month"] = eventsData["timestamp"].map(lambda x: x.month)
eventsData["day"] = eventsData["timestamp"].map(lambda x: x.day)


In [5]:
eventsData = eventsData.dropna(subset = ["model"] ,how = "all")

In [6]:
eventsData["marca"] = eventsData["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])

In [7]:
eventsData["version"] = eventsData["model"].map(lambda x:" ".join(x.split(" ")[0:2]) if(x.split(" ")[0] in ["iphone","ipad"]) else " ".join(x.split(" ")[1:3]))

In [8]:
eventsData.columns

Index(['Unnamed: 0', 'timestamp', 'event', 'person', 'url', 'sku', 'model',
       'condition', 'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'Parte del dia', 'month',
       'day', 'marca', 'version'],
      dtype='object')

In [9]:
eventsData["count"] = 1

In [10]:
actividad  = eventsData[["person","count"]].groupby("person",as_index =False).sum()

In [11]:

actividad.head()

,person,count
0,0008ed71,3
1,00091926,389
2,00091a7a,4
3,000ba417,161
4,000c79fe,4


In [12]:
eventsData = pd.merge(eventsData,actividad,how="left",on = "person")

In [13]:
eventsData.columns

Index(['Unnamed: 0', 'timestamp', 'event', 'person', 'url', 'sku', 'model',
       'condition', 'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'Parte del dia', 'month',
       'day', 'marca', 'version', 'count_x', 'count_y'],
      dtype='object')

In [14]:
eventsData.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,screen_resolution,operating_system_version,browser_version,Parte del dia,month,day,marca,version,count_x,count_y
0,0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,samsung galaxy j7 prime,Excelente,32GB,Dourado,...,360x640,Android 7,Chrome Mobile 64.0,Noche,5,18,samsung,galaxy j7,1,5
1,1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iphone 5s,Muito Bom,32GB,Cinza espacial,...,360x640,Android 5.1.1,Chrome Mobile 66.0,Noche,5,18,apple,iphone 5s,1,26
2,2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iphone 6s,Muito Bom,64GB,Prateado,...,360x640,Android 6.0.1,Chrome Mobile 61.0,Noche,5,18,apple,iphone 6s,1,439
3,3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iphone 7,Bom,128GB,Vermelho,...,1360x768,Windows 8.1,Chrome 66.0,Noche,5,18,apple,iphone 7,1,14
4,4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,lg g4 h818p,Excelente,32GB,Branco,...,1536x864,Windows 7,Chrome 66.0,Noche,5,18,lg,g4 h818p,1,754


In [15]:
eventsData["activo"] = eventsData["count_y"] > 7

In [16]:
marcas_ventas = eventsData[eventsData["event"] == "conversion"]["marca"].value_counts()

In [17]:
marcas_ventas

samsung     3615
apple       2031
motorola    1104
lg           161
lenovo        99
sony          56
asus          22
quantum        3
Name: marca, dtype: int64

In [18]:
version_ventas = eventsData[eventsData["event"] == "conversion"]["version"].value_counts()

In [19]:
version_ventas = version_ventas < 200

In [ ]:
marcas_ventas = marcas_ventas < 1000

In [ ]:
eventsData["marcas_malas"] = eventsData["marca"].map(lambda x: marcas_ventas[x] if(x in marcas_ventas.keys()) else True)

In [ ]:
eventsData["versiones_malas"] = eventsData["version"].map(lambda x: version_ventas[x] if(x in version_ventas.keys()) else True)

In [ ]:
eventsData["versiones_malas"].value_counts()

In [ ]:
eventsData = eventsData[["person","versiones_malas","marcas_malas","activo"]]

In [ ]:
eventsData = eventsData.groupby("person",as_index = False).agg({"versiones_malas":np.mean, "marcas_malas":np.mean, "activo":max})

In [ ]:
train = pd.read_csv("setEntrenamiento.csv")

In [ ]:
train = train.drop("Unnamed: 0",axis = 1)

In [ ]:
train = pd.merge(train,eventsData,on="person",how ="left")

In [ ]:
train["versiones_malas"] = train["versiones_malas"].fillna(train["versiones_malas"].mean())

In [ ]:
train["marcas_malas"] = train["marcas_malas"].fillna(train["marcas_malas"].mean())

In [ ]:
train["activo"] = train["activo"].fillna(False)

In [ ]:
train.to_csv("setEntrenamiento.csv",index = False)

In [ ]:
test = pd.read_csv("setTest.csv")

In [218]:
test = pd.merge(test,eventsData,on="person",how="left")

In [219]:
test.columns

Index(['person', 'cant_seciones', 'event_checkout', 'event_conversion',
       'event_ad campaign hit', 'event_lead', 'skus', 'screen_resolution',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'otros',
       'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana',
       'Parte del dia_Noche', 'Parte del dia_Tarde', 'campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse', 'channel_Direct

In [221]:
test["versiones_malas"] = test["versiones_malas"].fillna(test["versiones_malas"].mean())

In [222]:
test["marcas_malas"] = test["marcas_malas"].fillna(test["marcas_malas"].mean())

In [223]:
test["activo"] = test["activo"].fillna(False)

In [224]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19415 entries, 0 to 19414
Data columns (total 98 columns):
person                               19415 non-null object
cant_seciones                        19415 non-null int64
event_checkout                       19415 non-null float64
event_conversion                     19415 non-null float64
event_ad campaign hit                19415 non-null float64
event_lead                           19415 non-null float64
skus                                 19415 non-null int64
screen_resolution                    19415 non-null float64
Busca_terminos_populares             19415 non-null int64
Computer                             19415 non-null float64
Smartphone                           19415 non-null float64
otros                                19415 non-null float64
marca_apple                          19415 non-null float64
marca_asus                           19415 non-null float64
marca_lenovo                         19415 non-null float6

In [225]:
test.to_csv("setTest.csv",index = False)